In [1]:
import torch
from modelscope import AutoModel
from PIL import Image

MODEL_NAME= "/home/public/dkx/model/BAAI/BGE-VL-v1.5-zs"

model = AutoModel.from_pretrained(MODEL_NAME, trust_remote_code=True)
model.eval()
model.cuda()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LLaVANextForEmbedding(
  (model): LlavaNextModel(
    (vision_tower): CLIPVisionModel(
      (vision_model): CLIPVisionTransformer(
        (embeddings): CLIPVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
          (position_embedding): Embedding(577, 1024)
        )
        (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (encoder): CLIPEncoder(
          (layers): ModuleList(
            (0-23): 24 x CLIPEncoderLayer(
              (self_attn): CLIPAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
              )
              (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
           

In [2]:
with torch.no_grad():
    model.set_processor(MODEL_NAME)

    # query_inputs = model.data_process(
    #     text="Make the background dark, as if the camera has taken the photo at night",
    #     images="./assets/cir_query.png",
    #     q_or_c="q",
    #     task_instruction="Retrieve the target image that best meets the combined criteria by using both the provided image and the image retrieval instructions: "
    # )

    candidate_inputs = model.data_process(
        text="""
{"id":10093,"image_url":"/home/dkx/RAG4Ghart/bar/10093.png","type":"bar","theme":"Science and Engineering","title":"Data on R&D investment across cities","distribution":"random","display":"horizontal","header":"City,million USD(million USD)","data":[{"Crop category":"Rapeseed","metric tons":20000},{"Crop category":"Wheat","metric tons":2694},{"Crop category":"Apple","metric tons":2488},{"Crop category":"Sugarcane","metric tons":7573}]}
""",
        images=["/home/dukaixing/RAG4Ghart/Dataset-ZXQ/sample100/bar/png/21.png"],
        q_or_c="c",
    )

    # query_embs = model(**query_inputs, output_hidden_states=True)[:, -1, :]
    candi_embs = model(**candidate_inputs, output_hidden_states=True)[:, -1, :]

    # query_embs = torch.nn.functional.normalize(query_embs, dim=-1)
    candi_embs = torch.nn.functional.normalize(candi_embs, dim=-1)

    # scores = torch.matmul(query_embs, candi_embs.T)
print(candi_embs.shape)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:382: masked_scatter_size_check: block: [0,0,0], thread: [0,0,0] Assertion `totalElements <= srcSize` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [4]:
with torch.no_grad():
    # query = model.encode(
    #     images = "./assets/cir_query.png",
    #     text = "Make the background dark, as if the camera has taken the photo at night"
    # )

    candidates = model.encode(
        text = """
{"id":10093,"image_url":"/home/dkx/RAG4Ghart/bar/10093.png","type":"bar","theme":"Science and Engineering","title":"Data on R&D investment across cities","distribution":"random","display":"horizontal","header":"City,million USD(million USD)","data":[{"Crop category":"Rapeseed","metric tons":20000},{"Crop category":"Wheat","metric tons":2694},{"Crop category":"Apple","metric tons":2488},{"Crop category":"Sugarcane","metric tons":7573}]}
""",
        images = ["/home/dukaixing/RAG4Ghart/Dataset-ZXQ/sample100/bar/png/21.png"]
    )

    # scores = query @ candidates.T

# print(query)
print(candidates.shape)
# print(scores)

AttributeError: 'LLaVANextForEmbedding' object has no attribute 'encode'

In [2]:
import torch

a = [torch.ones(2880, 4096)]
b = torch.cat(a, dim=0)
print(b.shape)

torch.Size([2880, 4096])
